### Тема «POS-tagger и NER»

**Задание 1. Написать теггер на данных с русским языком.**

1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации.
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов.
3. сравнить все реализованные методы, сделать выводы.


**Задание 2. Проверить, насколько хорошо работает NER.**

1. проверить NER из nltk/spacy/deeppavlov.

2. написать свой NER, попробовать разные подходы.

*a. передаём в сетку токен и его соседей.*

*b. передаём в сетку только токен.*

*c. свой вариант.*

3. сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.

### Задание 1.

**Загрузим необходимые библиотеки и данные.**

In [1]:
!pip install pyconll

In [2]:
!wget -O ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2023-10-16 10:20:06--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736581 (39M) [text/plain]
Saving to: ‘ru_syntagrus-ud-train.conllu’

ru_syntagrus-ud-tra 100%[===================>]  38.85M   200MB/s    in 0.2s    

2023-10-16 10:20:07 (200 MB/s) - ‘ru_syntagrus-ud-train.conllu’ saved [40736581/40736581]

--2023-10-16 10:20:07--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108

In [3]:
import pandas as pd
import numpy as np
import re

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import pyconll
import nltk
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger, RegexpTagger

import warnings
warnings.filterwarnings("ignore")

In [4]:
data_train = pyconll.load_from_file('ru_syntagrus-ud-train.conllu')
data_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [5]:
fdata_train = []
for sent in data_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])

fdata_test = []
for sent in data_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])

fdata_sent_test = []
for sent in data_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [6]:
len(fdata_train), len(fdata_test), len(fdata_sent_test)

(24516, 8906, 8906)

In [7]:
fdata_train[:2]

[[('Анкета', 'NOUN'), ('.', 'PUNCT')],
 [('Начальник', 'NOUN'),
  ('областного', 'ADJ'),
  ('управления', 'NOUN'),
  ('связи', 'NOUN'),
  ('Семен', 'PROPN'),
  ('Еремеевич', 'PROPN'),
  ('был', 'AUX'),
  ('человек', 'NOUN'),
  ('простой', 'ADJ'),
  (',', 'PUNCT'),
  ('приходил', 'VERB'),
  ('на', 'ADP'),
  ('работу', 'NOUN'),
  ('всегда', 'ADV'),
  ('вовремя', 'ADV'),
  (',', 'PUNCT'),
  ('здоровался', 'VERB'),
  ('с', 'ADP'),
  ('секретаршей', 'NOUN'),
  ('за', 'ADP'),
  ('руку', 'NOUN'),
  ('и', 'CCONJ'),
  ('иногда', 'ADV'),
  ('даже', 'PART'),
  ('писал', 'VERB'),
  ('в', 'ADP'),
  ('стенгазету', 'NOUN'),
  ('заметки', 'NOUN'),
  ('под', 'ADP'),
  ('псевдонимом', 'NOUN'),
  ('"', 'PUNCT'),
  ('Муха', 'NOUN'),
  ('"', 'PUNCT'),
  ('.', 'PUNCT')]]

**Проверим работу всех теггеров поочередно.**

In [8]:
default_tagger = nltk.DefaultTagger('NN')
default_acc = default_tagger.evaluate(fdata_test)

unigram_tagger = UnigramTagger(fdata_train)
unigram_acc = unigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train)
bigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train)
trigram_acc = trigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_unigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_bigram_unigram_acc = trigram_tagger.evaluate(fdata_test)

print(f'Accuracy:\nDefault Tagger: {round(default_acc, 3)},\nUnigram Tagger: {round(unigram_acc, 3)},\nBigram Tagger: {round(bigram_acc, 5)},\n'
      f'Trigram Tagger: {round(trigram_acc, 3)},\nBigram and Unigram Tagger: {round(bigram_unigram_acc, 5)},\n'
      f'Trigram, Bigram and Unigram Tagger: {round(trigram_bigram_unigram_acc, 5)},\n')

Accuracy:
Default Tagger: 0.0,
Unigram Tagger: 0.824,
Bigram Tagger: 0.60939,
Trigram Tagger: 0.178,
Bigram and Unigram Tagger: 0.82928,
Trigram, Bigram and Unigram Tagger: 0.82914,



**Различные комбинации теггеров могут давать прирост качества.**

**Для эксперимента попробуем объединить работу всех теггеров с помощью функции.**

In [9]:
def union_taggers(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN')
tag = union_taggers(fdata_train,
                     [UnigramTagger, BigramTagger, TrigramTagger],
                     backoff = backoff)

tag.evaluate(fdata_test)

0.827905462595221

**Получили некий усредненный результат. Не самый высокий.**
**ВЫВОД: на каждом корпусе пробовать все варианты и выбирать наилучший.**

### Попробуем написать теггер.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder

**Преобразуем тренировочный датасет в списки слов и списки POS-разметки.**

In [11]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])

test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(' ' if tok[0] is None else tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [12]:
train_tok[:7], train_label[:7]

(['Анкета', '.', 'Начальник', 'областного', 'управления', 'связи', 'Семен'],
 ['NOUN', 'PUNCT', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'PROPN'])

In [13]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
train_enc_labels

array([ 7, 13,  7, ...,  1, 11, 13])

In [14]:
test_enc_labels = le.transform(test_label)
test_enc_labels

array([ 7, 13,  1, ...,  0,  7, 13])

In [15]:
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [16]:
%time

vectorizers = [CountVectorizer(ngram_range=(1, 5), analyzer='char'),
               TfidfVectorizer(ngram_range=(1, 5), analyzer='char'),
               HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=1000)]
vectorizers_word = [CountVectorizer(ngram_range=(1, 5), analyzer='word'),
               TfidfVectorizer(ngram_range=(1, 5), analyzer='word'),
               HashingVectorizer(ngram_range=(1, 5), analyzer='word', n_features=1000)]
n_features = [2000, 3000, 5000]
vectorizers_hash = [HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=feat) for feat in n_features]
vectorizers_hash_word = [HashingVectorizer(ngram_range=(1, 5), analyzer='word', n_features=feat) for feat in n_features]
f1_scores = []
accuracy_scores = []

for vectorizer in vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word:
    X_train = vectorizer.fit_transform(train_tok)
    X_test = vectorizer.transform(test_tok)

    lr = LogisticRegression(random_state=0, max_iter=100)
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)
    f1 = f1_score(test_enc_labels, pred, average='weighted')
    f1_scores.append(f1)
    acc = accuracy_score(test_enc_labels, pred)
    accuracy_scores.append(acc)

    print(vectorizer)
    print(classification_report(test_enc_labels, pred, target_names=le.classes_))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.77 µs
CountVectorizer(analyzer='char', ngram_range=(1, 5))
              precision    recall  f1-score   support

         ADJ       0.94      0.92      0.93     15103
         ADP       0.98      1.00      0.99     13717
         ADV       0.91      0.93      0.92      7783
         AUX       0.82      0.96      0.88      1390
       CCONJ       0.89      0.97      0.93      5672
         DET       0.83      0.79      0.81      4265
        INTJ       0.39      0.29      0.33        24
        NOUN       0.94      0.97      0.96     36238
      NO_TAG       1.00      0.77      0.87       265
         NUM       0.86      0.89      0.88      1734
        PART       0.95      0.77      0.85      5125
        PRON       0.90      0.84      0.87      7444
       PROPN       0.84      0.66      0.73      5473
       PUNCT       1.00      1.00      1.00     29186
       SCONJ       0.75      0.97      0.85      2865
         SYM      

**Для удобства представим данные в виде таблицы:**

In [17]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [18]:
result_model = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'f1_score': f1_scores})
result_model.sort_values('f1_score', ascending=False)

,Vectorizer,f1_score
0,"CountVectorizer(analyzer='char', ngram_range=(1, 5))",0.938119
1,"TfidfVectorizer(analyzer='char', ngram_range=(1, 5))",0.926643
8,"HashingVectorizer(analyzer='char', n_features=5000, ngram_range=(1, 5))",0.909308
7,"HashingVectorizer(analyzer='char', n_features=3000, ngram_range=(1, 5))",0.898656
6,"HashingVectorizer(analyzer='char', n_features=2000, ngram_range=(1, 5))",0.892840
2,"HashingVectorizer(analyzer='char', n_features=1000, ngram_range=(1, 5))",0.866909
3,"CountVectorizer(ngram_range=(1, 5))",0.657903
4,"TfidfVectorizer(ngram_range=(1, 5))",0.650296
11,"HashingVectorizer(n_features=5000, ngram_range=(1, 5))",0.584730
10,"HashingVectorizer(n_features=3000, ngram_range=(1, 5))",0.567713


In [19]:
result_model_acc = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'Accuracy': accuracy_scores})
result_model_acc.sort_values('Accuracy', ascending=False)

,Vectorizer,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(1, 5))",0.939469
1,"TfidfVectorizer(analyzer='char', ngram_range=(1, 5))",0.928986
8,"HashingVectorizer(analyzer='char', n_features=5000, ngram_range=(1, 5))",0.912540
7,"HashingVectorizer(analyzer='char', n_features=3000, ngram_range=(1, 5))",0.902181
6,"HashingVectorizer(analyzer='char', n_features=2000, ngram_range=(1, 5))",0.896243
2,"HashingVectorizer(analyzer='char', n_features=1000, ngram_range=(1, 5))",0.870734
3,"CountVectorizer(ngram_range=(1, 5))",0.649248
4,"TfidfVectorizer(ngram_range=(1, 5))",0.640016
11,"HashingVectorizer(n_features=5000, ngram_range=(1, 5))",0.601947
10,"HashingVectorizer(n_features=3000, ngram_range=(1, 5))",0.592669


### В результате эксперимента проверили различные векторайзеры для разного количества символов. Наилучший результат показали символьные N-граммы. Результат N-грамм для слов оказался значительно хуже.

### Задание 2.

In [20]:
!pip install corus

In [21]:
!pip install razdel
!pip install -U spacy
!python -m spacy info

2023-10-16 10:36:38.477188: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-16 10:36:41.595379: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)

============================== Info about spaCy ==============================

spaCy version    3.7.1                      

### NLTK

In [22]:
import nltk
from nltk.tokenize import word_tokenize
import matplotlib
import pyconll
import corus
from corus import load_ne5
import re
import spacy
from spacy import displacy

In [23]:
import nltk
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')

[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
nltk.help.upenn_tagset('RB')
nltk.help.upenn_tagset('NN')
nltk.help.upenn_tagset('VB')

RB: adverb
    occasionally unabatingly maddeningly adventurously professedly
    stirringly prominently technologically magisterially predominately
    swiftly fiscally pitilessly ...
NN: noun, common, singular or mass
    common-carrier cabbage knuckle-duster Casino afghan shed thermostat
    investment slide humour falloff slick wind hyena override subhumanity
    machinist ...
VB: verb, base form
    ask assemble assess assign assume atone attention avoid bake balkanize
    bank begin behold believe bend benefit bevel beware bless boil bomb
    boost brace break bring broil brush build ...


In [25]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2023-10-16 10:37:44--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip.3’

collection5.zip.3   100%[===================>]   1.81M  1.90MB/s    in 1.0s    

2023-10-16 10:37:46 (1.90 MB/s) - ‘collection5.zip.3’ saved [1899530/1899530]



In [26]:
!unzip collection5.zip

Archive:  collection5.zip
replace Collection5/001.ann? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection5/006.txt     
  inflating: Collection5/007.ann     
  inflating: Collection5/007.txt     
  inflating: Collection5/008.ann     
  inflating: Collection5/008.txt     
  inflating: Collection5/009.ann     
  inflating: Collection5/009.txt     
  inflating: Collection5/010.ann     
  inflating: Collection5/010.txt     
  inflating: Collection5/011.ann     
  inflating: Collection5/011.txt     
  inflating: Collection5/012.ann     
  inflating: Collection5/012.tx

In [27]:
records = load_ne5('Collection5/')

In [28]:
document = next(records)
text = document.text
# text
text = re.sub('\r\n\r\n',' ',text)
text = re.sub('\r\n',' ',text)
text

'"Единая Россия" предложила 12 новых председателей комитетов Госдумы из своих 15. Партия "Единая Россия" предложила 12 новых председателей комитетов Государственной думы VI созыва из своих 15. Об этом журналистам сообщил секретарь президиума генерального совета "Единой России" Сергей Неверов. По его словам, на пост главы комитета по конституционному законодательству и государственному строительству предложена кандидатура Владимира Плигина (сохранил пост), комитета по гражданскому, процессуальному, уголовному и арбитражному законодательству - Павла Крашенинникова (сохранил пост), комитета по труду и социальной политики и делам ветеранов - Андрея Исаева (сохранил пост); по бюджету и налогам - Андрея Макарова, по финансовому рынку - Натальи Бурыкиной; по экономической политике, инновационному развитию и предпринимательству - Игоря Руденского; по федеративному устройству - Виктора Кидяева, по регламенту и организации работы Госдумы - Ильдара Габдрахманова, по делам национальностей - Гаджим

In [29]:
document.text = text

In [30]:
nltk.pos_tag(nltk.word_tokenize(text)) # Как предварительно очистить все статьи в словаре.

[('``', '``'),
 ('Единая', 'NN'),
 ('Россия', 'NN'),
 ("''", "''"),
 ('предложила', '$'),
 ('12', 'CD'),
 ('новых', 'NNP'),
 ('председателей', 'NNP'),
 ('комитетов', 'NNP'),
 ('Госдумы', 'NNP'),
 ('из', 'NNP'),
 ('своих', 'VBD'),
 ('15', 'CD'),
 ('.', '.'),
 ('Партия', 'VB'),
 ('``', '``'),
 ('Единая', 'JJ'),
 ('Россия', 'NN'),
 ("''", "''"),
 ('предложила', '$'),
 ('12', 'CD'),
 ('новых', 'NNP'),
 ('председателей', 'NNP'),
 ('комитетов', 'NNP'),
 ('Государственной', 'NNP'),
 ('думы', 'NNP'),
 ('VI', 'NNP'),
 ('созыва', 'NNP'),
 ('из', 'NNP'),
 ('своих', 'VBD'),
 ('15', 'CD'),
 ('.', '.'),
 ('Об', 'VB'),
 ('этом', 'JJ'),
 ('журналистам', 'NNP'),
 ('сообщил', 'NNP'),
 ('секретарь', 'NNP'),
 ('президиума', 'NNP'),
 ('генерального', 'NNP'),
 ('совета', 'NNP'),
 ('``', '``'),
 ('Единой', 'FW'),
 ('России', 'NN'),
 ("''", "''"),
 ('Сергей', 'NN'),
 ('Неверов', 'NNP'),
 ('.', '.'),
 ('По', 'VB'),
 ('его', 'JJ'),
 ('словам', 'NNP'),
 (',', ','),
 ('на', 'NNP'),
 ('пост', 'NNP'),
 ('главы', 'N

In [ ]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))) if hasattr(chunk, 'label') }

{('Алексея', 'GPE'),
 ('Андрея', 'GPE'),
 ('Бурматова', 'PERSON'),
 ('Бурыкиной', 'ORGANIZATION'),
 ('Виктора', 'GPE'),
 ('Владимира', 'GPE'),
 ('Габдрахманова', 'PERSON'),
 ('Гаджимеда', 'GPE'),
 ('Говорухина', 'ORGANIZATION'),
 ('Госдумы', 'PERSON'),
 ('Евгения', 'GPE'),
 ('Игоря', 'GPE'),
 ('Ильдара', 'GPE'),
 ('Ирины', 'GPE'),
 ('Исаева', 'GPE'),
 ('Кидяева', 'PERSON'),
 ('ЛДПР', 'ORGANIZATION'),
 ('Макарова', 'PERSON'),
 ('Москвичева', 'ORGANIZATION'),
 ('Натальи', 'GPE'),
 ('Николая', 'GPE'),
 ('Павла', 'GPE'),
 ('Панкова', 'PERSON'),
 ('Пушкова', 'PERSON'),
 ('Россия', 'PERSON'),
 ('Руденского', 'ORGANIZATION'),
 ('Сафаралиева', 'PERSON'),
 ('Сергей Неверов', 'PERSON'),
 ('Станислава', 'GPE'),
 ('Яровой', 'PERSON')}

**В целом - работает. Но ошибки встречаются ('Россиия', 'PERSON').**

In [31]:
document.spans

[Ne5Span(
     index='T1',
     type='ORG',
     start=1,
     stop=14,
     text='Единая Россия'
 ),
 Ne5Span(
     index='T2',
     type='ORG',
     start=60,
     stop=67,
     text='Госдумы'
 ),
 Ne5Span(
     index='T3',
     type='ORG',
     start=92,
     stop=105,
     text='Единая Россия'
 ),
 Ne5Span(
     index='T4',
     type='ORG',
     start=151,
     stop=171,
     text='Государственной думы'
 ),
 Ne5Span(
     index='T5',
     type='ORG',
     start=265,
     stop=278,
     text='Единой России'
 ),
 Ne5Span(
     index='T6',
     type='PER',
     start=280,
     stop=294,
     text='Сергей Неверов'
 ),
 Ne5Span(
     index='T7',
     type='PER',
     start=430,
     stop=447,
     text='Владимира Плигина'
 ),
 Ne5Span(
     index='T8',
     type='PER',
     start=553,
     stop=573,
     text='Павла Крашенинникова'
 ),
 Ne5Span(
     index='T9',
     type='PER',
     start=651,
     stop=664,
     text='Андрея Исаева'
 ),
 Ne5Span(
     index='T10',
     type='PER',
   

### SpaCy

In [32]:
!python -m spacy download ru_core_news_sm

2023-10-16 10:38:32.515460: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 56.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [33]:
import spacy
from spacy import displacy
import ru_core_news_sm
from spacy.lang.ru.examples import sentences
from spacy.lang.ru import Russian

In [34]:
nlp = spacy.load("ru_core_news_sm")

In [35]:
ny_bb = text
article = nlp(ny_bb)

In [36]:
displacy.render(article, jupyter=True, style='ent')

**На этом тексте библиотека SpaCy без ошибок.**

**Посмотрим на список токенов, частей речи и сущностей.**

In [37]:
for token in article:
    print(token.text, token.pos_, token.dep_)

" PUNCT punct
Единая ADJ amod
Россия PROPN nsubj
" PUNCT punct
предложила VERB ROOT
12 NUM nummod
новых ADJ amod
председателей NOUN obj
комитетов NOUN nmod
Госдумы PROPN nmod
из ADP case
своих DET det
15 NUM nummod
. PUNCT punct
Партия NOUN nsubj
" PUNCT punct
Единая ADJ amod
Россия PROPN appos
" PUNCT punct
предложила VERB ROOT
12 NUM nummod
новых ADJ amod
председателей NOUN obj
комитетов NOUN nmod
Государственной ADJ amod
думы NOUN nmod
VI ADJ amod
созыва NOUN nmod
из ADP case
своих DET det
15 NUM nummod
. PUNCT punct
Об ADP case
этом PRON obl
журналистам NOUN iobj
сообщил VERB ROOT
секретарь NOUN nsubj
президиума NOUN nmod
генерального ADJ amod
совета NOUN nmod
" PUNCT punct
Единой ADJ amod
России PROPN nmod
" PUNCT punct
Сергей PROPN appos
Неверов PROPN flat:name
. PUNCT punct
По ADP case
его DET det
словам NOUN parataxis
, PUNCT punct
на ADP case
пост NOUN obl
главы NOUN nmod
комитета NOUN nmod
по ADP case
конституционному ADJ amod
законодательству NOUN nmod
и CCONJ cc
государстве

### DeepPavlov

In [38]:
!pip install pymorphy2 #==0.9

In [39]:
# !pip uninstall -y tensorflow tensorflow-gpu
!pip install numpy scipy librosa unidecode inflect librosa transformers
!pip install deeppavlov

In [40]:
text

'"Единая Россия" предложила 12 новых председателей комитетов Госдумы из своих 15. Партия "Единая Россия" предложила 12 новых председателей комитетов Государственной думы VI созыва из своих 15. Об этом журналистам сообщил секретарь президиума генерального совета "Единой России" Сергей Неверов. По его словам, на пост главы комитета по конституционному законодательству и государственному строительству предложена кандидатура Владимира Плигина (сохранил пост), комитета по гражданскому, процессуальному, уголовному и арбитражному законодательству - Павла Крашенинникова (сохранил пост), комитета по труду и социальной политики и делам ветеранов - Андрея Исаева (сохранил пост); по бюджету и налогам - Андрея Макарова, по финансовому рынку - Натальи Бурыкиной; по экономической политике, инновационному развитию и предпринимательству - Игоря Руденского; по федеративному устройству - Виктора Кидяева, по регламенту и организации работы Госдумы - Ильдара Габдрахманова, по делам национальностей - Гаджим

In [41]:
from deeppavlov import build_model
ner_model = build_model('ner_ontonotes_bert', download=True, install=True)
ner_model(text)

2023-10-16 10:42:51.762 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/v1/ner/ner_ontonotes_bert_torch_crf.tar.gz download because of matching hashes
INFO:deeppavlov.download:Skipped http://files.deeppavlov.ai/v1/ner/ner_ontonotes_bert_torch_crf.tar.gz download because of matching hashes
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Ber

[[['"'],
  ['Е'],
  ['д'],
  ['и'],
  ['н'],
  ['а'],
  ['я'],
  [],
  ['Р'],
  ['о'],
  ['с'],
  ['с'],
  ['и'],
  ['я'],
  ['"'],
  [],
  ['п'],
  ['р'],
  ['е'],
  ['д'],
  ['л'],
  ['о'],
  ['ж'],
  ['и'],
  ['л'],
  ['а'],
  [],
  ['1'],
  ['2'],
  [],
  ['н'],
  ['о'],
  ['в'],
  ['ы'],
  ['х'],
  [],
  ['п'],
  ['р'],
  ['е'],
  ['д'],
  ['с'],
  ['е'],
  ['д'],
  ['а'],
  ['т'],
  ['е'],
  ['л'],
  ['е'],
  ['й'],
  [],
  ['к'],
  ['о'],
  ['м'],
  ['и'],
  ['т'],
  ['е'],
  ['т'],
  ['о'],
  ['в'],
  [],
  ['Г'],
  ['о'],
  ['с'],
  ['д'],
  ['у'],
  ['м'],
  ['ы'],
  [],
  ['и'],
  ['з'],
  [],
  ['с'],
  ['в'],
  ['о'],
  ['и'],
  ['х'],
  [],
  ['1'],
  ['5'],
  ['.'],
  [],
  ['П'],
  ['а'],
  ['р'],
  ['т'],
  ['и'],
  ['я'],
  [],
  ['"'],
  ['Е'],
  ['д'],
  ['и'],
  ['н'],
  ['а'],
  ['я'],
  [],
  ['Р'],
  ['о'],
  ['с'],
  ['с'],
  ['и'],
  ['я'],
  ['"'],
  [],
  ['п'],
  ['р'],
  ['е'],
  ['д'],
  ['л'],
  ['о'],
  ['ж'],
  ['и'],
  ['л'],
  ['а'],
  [],
  ['1'],
 

**Написать свой NER**

In [42]:
from sklearn.metrics import confusion_matrix, classification_report,f1_score, accuracy_score

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from razdel import tokenize
from corus import load_ne5

In [43]:
def get_classification_report(y_test_true, y_test_pred):
    print(classification_report(y_test_true, y_test_pred))

    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

Воспользуемся размеченным корпусом текстов.

In [44]:
records = load_ne5('Collection5/')
next(records)

Ne5Markup(
    id='487',
    text='"Единая Россия" предложила 12 новых председателей комитетов Госдумы из своих 15.\r\n\r\nПартия "Единая Россия" предложила 12 новых председателей комитетов Государственной думы VI созыва из своих 15. Об этом журналистам сообщил секретарь президиума генерального совета "Единой России" Сергей Неверов.\r\n\r\nПо его словам, на пост главы комитета по конституционному законодательству и государственному строительству предложена кандидатура Владимира Плигина (сохранил пост), комитета по гражданскому, процессуальному, уголовному и арбитражному законодательству - Павла Крашенинникова (сохранил пост), комитета по труду и социальной политики и делам ветеранов - Андрея Исаева (сохранил пост); по бюджету и налогам - Андрея Макарова, по финансовому рынку - Натальи Бурыкиной; по экономической политике, инновационному развитию и предпринимательству - Игоря Руденского; по федеративному устройству - Виктора Кидяева, по регламенту и организации работы Госдумы - Ильдара 

In [45]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [46]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [47]:
df_words['tag'].value_counts()

OUT         219044
PER          21165
ORG          13637
LOC           4568
GEOPOLIT      4356
MEDIA         2482
Name: tag, dtype: int64

In [48]:
df_words.head()

,word,tag
0,Оппозиция,OUT
1,обвинила,OUT
2,президента,OUT
3,Египта,GEOPOLIT
4,в,OUT


In [49]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

**Закодируем целевую переменную**

In [50]:
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

Посмотрим на классы

In [51]:
encoder.classes_

array(['GEOPOLIT', 'LOC', 'MEDIA', 'ORG', 'OUT', 'PER'], dtype=object)

In [52]:
train_x.apply(len).max(axis=0)

32

In [53]:
valid_x

175295               С
45229                с
200532              из
63124           Вадиму
2919       инициативой
              ...     
113609               -
214226     организаций
23602              При
56684     департамента
166825           будут
Name: word, Length: 66313, dtype: object

In [54]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [55]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [56]:
def custom_standardization(input_data):
    # Здесь может быть предобработка текста
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization(
                            standardize=custom_standardization,
                            max_tokens=vocab_size,
                            output_mode='int',
                            #ngrams=(1, 3),
                            output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [57]:
len(vectorize_layer.get_vocabulary())

29941

In [58]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax') # [OUT, PER, ORG, LOC, GEOPOLIT, MEDIA]

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)

        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)

        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [59]:
mmodel = modelNER()

In [60]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [61]:
mmodel.fit( train_data,
            validation_data=valid_data,
            epochs=3)

Epoch 1/3
12434/12434 [==============================] - 148s 12ms/step - loss: 0.2950 - accuracy: 0.9144 - val_loss: 0.2060 - val_accuracy: 0.9389
Epoch 2/3
12434/12434 [==============================] - 140s 11ms/step - loss: 0.1257 - accuracy: 0.9625 - val_loss: 0.2392 - val_accuracy: 0.8953
Epoch 3/3
12434/12434 [==============================] - 137s 11ms/step - loss: 0.1096 - accuracy: 0.9653 - val_loss: 0.2705 - val_accuracy: 0.8880


In [62]:
pred_y = mmodel.predict(valid_x)
y_pred_classes = np.argmax(pred_y,axis=1)

2073/2073 [==============================] - 4s 2ms/step


In [63]:
f1 = f1_score(valid_y, y_pred_classes, average= "weighted")
f1

0.9034458143947679

In [64]:
print(f"Classes: {encoder.classes_}\r\n")

get_classification_report(valid_y, y_pred_classes)

Classes: ['GEOPOLIT' 'LOC' 'MEDIA' 'ORG' 'OUT' 'PER']

              precision    recall  f1-score   support

           0       0.88      0.91      0.90      1085
           1       0.87      0.78      0.82      1127
           2       0.93      0.77      0.84       619
           3       0.30      0.68      0.42      3387
           4       0.97      0.92      0.94     54910
           5       0.98      0.71      0.82      5185

    accuracy                           0.89     66313
   macro avg       0.82      0.79      0.79     66313
weighted avg       0.93      0.89      0.90     66313

CONFUSION MATRIX

col_0    0    1    2     3      4     5
row_0                                  
0      987   19    1    69      9     0
1       19  879    3   174     50     2
2        9    6  475    62     67     0
3       90   29   17  2292    942    17
4       10   75   15  4148  50600    62
5        1    0    0   895    633  3656


#### Обучим нейронную сеть на биграммах и триграммах.

In [65]:
def custom_standardization(input_data):
    # Здесь может быть предобработка текста.
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization(
                            standardize=custom_standardization,
                            max_tokens=vocab_size,
                            output_mode='int',
                            ngrams=(1, 3),
                            output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [66]:
mmodel = modelNER()

In [67]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [68]:
mmodel.fit( train_data,
            validation_data=valid_data,
            epochs=3)

Epoch 1/3
12434/12434 [==============================] - 140s 11ms/step - loss: 0.2952 - accuracy: 0.9142 - val_loss: 0.2048 - val_accuracy: 0.9397
Epoch 2/3
12434/12434 [==============================] - 139s 11ms/step - loss: 0.1255 - accuracy: 0.9625 - val_loss: 0.2473 - val_accuracy: 0.8951
Epoch 3/3
12434/12434 [==============================] - 143s 12ms/step - loss: 0.1091 - accuracy: 0.9654 - val_loss: 0.2082 - val_accuracy: 0.9416


In [69]:
pred_y = mmodel.predict(valid_x)
y_pred_classes = np.argmax(pred_y,axis=1)

2073/2073 [==============================] - 4s 2ms/step


In [70]:
f1 = f1_score(valid_y, y_pred_classes, average= "weighted")
f1

0.9372984811999705

In [71]:
print(f"Classes: {encoder.classes_}\r\n")

get_classification_report(valid_y, y_pred_classes)

Classes: ['GEOPOLIT' 'LOC' 'MEDIA' 'ORG' 'OUT' 'PER']

              precision    recall  f1-score   support

           0       0.89      0.91      0.90      1085
           1       0.87      0.78      0.82      1127
           2       0.93      0.76      0.84       619
           3       0.87      0.57      0.69      3387
           4       0.94      0.99      0.97     54910
           5       0.98      0.71      0.82      5185

    accuracy                           0.94     66313
   macro avg       0.91      0.79      0.84     66313
weighted avg       0.94      0.94      0.94     66313

CONFUSION MATRIX

col_0    0    1    2     3      4     5
row_0                                  
0      986   23    1    27     48     0
1       17  878    2    22    206     2
2        9    5  473    18    114     0
3       87   33   16  1920   1314    17
4       10   70   16   224  54528    62
5        1    0    0     2   1524  3658


### Результьат выше у сети, которая обучалась на N-граммах.